In [1]:
from ev_model import persistency, environment, elements
from ev_model.utilities import geometry as evmetry
import numpy as np
import copy
!pwd

/home/cmoralesmx/git/ev_diffusion_tools


In [2]:
def load_system_state(directory, file_name):
    # *grids contains [secretory, ciliary, all (secretory+ciliary), evs]
    [itno, environment, secretory, ciliary, evs, evs_with_errors, *grids] = persistency.read_xml(directory, file_name)
    print('load_system_state len(grids):', len(grids[0]))
    parameters = ['id', 'x', 'y', 'radius_um', 'age']
    evs_df = persistency.agent_data_to_data_frame(evs['dictionaries'], parameters)
    return itno, environment, secretory, ciliary, evs, evs_df, evs_with_errors, grids[0]

In [3]:
# offending ev and colliding cell states must be loaded for the moment before and at collision.
# This means, we must load the first xml file containing NaN of Inf values on X or Y
# and another xml immediately before that file.

# we must have the same functions used on the collision detection and collision resolution stages
# Instead of sampling random variables, we use the data loaded on the XML file

# ev 9423 collides with a secretory cell 931 at iteration 210523
e1 = {
    'base_path':'/home/cmoralesmx/ev_iters/v31/dead_newEv15s/isth/0/',
    'collision_at':210523,
    'ev_id':9423,
}

e2 = {
    'base_path':'/home/cmoralesmx/ev_iters/v31/dead_newEv15s/isth/0/',
    'collision_at': 243646,
    'ev_id': 7270,
}

e3 = {
    'base_path':'/home/cmoralesmx/ev_iters/v31/dead_newEv15s/isth/0/',
    'collision_at': 256111,
    'preceding_iterations_to_check': 1,
    'ev_id': 7462,
    'secretory_id' : 2329
}

In [4]:
def dotprod(x1, y1, x2, y2):
    return x1*x2 + y1*y2

def vlength(x, y):
    return np.sqrt(x*x + y*y)

def projection(a_x, a_y, b_x,  b_y):
    """
    Gives the length of the projection of vector A in the direction of vector B
    """
    length = vlength(a_x, a_y)
    length_vec = vlength(b_x, b_y)
    if length == 0 or length_vec == 0:
        return 0
    return (a_x * b_x + a_y * b_y) / length_vec;


def project(v1, v2):
    """
    Returns a vector parallel to v2 with the length of the projection of v2 in v1
    project: function(vec) {
        return vec.para(this.projection(vec));
    }
    """
    mag = projection(v1[0], v1[1], v2[0], v2[1])
    return parallel(v2, mag)

def add_scaled(src_x, src_y, vec_x, vec_y, scale):
    """
    Scales vector b by scale and adds it to vector a
    """
    return np.array([src_x + scale * vec_x, src_y + scale * vec_y])

def parallel(vec, magnitude, debug=False):
    """
    Gives a vector parallel to vector vec and of length magnitude
    """
    if debug:
        print(f'producing a parallel vector to vec:{vec}, by magnitude:{magnitude}')
    vector_length = vlength(vec[0], vec[1])
    if debug:
        print(f'vector_length: {vector_length}')
    return vec * (magnitude / vector_length)

In [5]:
def cell_collision_resolution(cell_id, agent, debug=False):
    """
    Expects two objects for checking if any collision would have been possible
    """
    cell = cells[cell_id]
    if debug:
        print(' > cell_collision_resolution() based on')
    
    # // vectors from ev to endpoints of wall
    ev_p1x = cell.p1[0] - agent.x
    ev_p1y = cell.p1[1] - agent.y
    ev_p2x = cell.p2[0] - agent.x
    ev_p2y = cell.p2[1] - agent.y
    if debug:
        print(f"  >>  ev_p1x: {ev_p1x:+.6f}, ev_p1y: {ev_p1y:+.6f}")
        print(f"  >>  ev_p2x: {ev_p2x:+.6f}, ev_p2y: {ev_p2y:+.6f}")
    
    #// projection of above vectors onto wall vector
    # var proj1 = ballp1.projection(wdir);
    proj1 = projection(ev_p1x, ev_p1y, cell.direction[0], cell.direction[1])
    proj2 = projection(ev_p2x, ev_p2y, cell.direction[0], cell.direction[1])
    if debug:
        print(f"  >>  projection1: {proj1:+.6f}, projection2: {proj2:+.6f}")

    #// Perpendicular distance vector from the object to the wall
    #// var dist = ballp1.addScaled(wdir.unit(), proj1*(-1));
    dist_x = ev_p1x + cell.direction_unit[0] * -proj1
    dist_y = ev_p1y + cell.direction_unit[1] * -proj1
    dist = add_scaled(ev_p1x, ev_p1y, cell.direction_unit[0], cell.direction_unit[1], -proj1)
    dist_length = vlength(dist_x, dist_y)
    
    if debug:
        print(f'  >>  dist_length: {dist_length:+.6f}, dist_x: {dist_x:+.6f}, dist_y: {dist_y:+.6f} | dist: ({dist[0]:+.6f}, {dist[1]:+.6f})')
    
    #// collision detection
    #//var test = ((Math.abs(proj1) < wdir.length()) && (Math.abs(proj2) < wdir.length()));
    
    test = (abs(proj1) < cell.direction_length) and (abs(proj2) < cell.direction_length)
    if debug:
        print(f'  >>  Should we test for a collision? {test}')
        print(f'  >>  Collision straight with the cell? (dist.length() < radius_um): {dist_length < agent.radius_um} && test')
        print(f'  >>  Collision with 1st edge? vlength(ev_p1x, ev_p1y): {vlength(ev_p1x, ev_p1y):+.6f} < radius? {vlength(ev_p1x, ev_p1y) < agent.radius_um}')
        print(f'  >>  Collision with 2nd edge? vlength(ev_p2x, ev_p2y): {vlength(ev_p2x, ev_p2y):+.6f} < radius? {vlength(ev_p2x, ev_p2y) < agent.radius_um}')

In [6]:
def cell_collision_recomputation(future_x, future_y, vx_1, vy_1, agent_radius_um, cell, agent, debug=False):
    """
    Expects:
    floats - future coordinates of the agent, the starting velocity, and radius
    objects: cell, agent
    Should return:
    int type - type of collision [0:no collision, 1:collision with segment, 2:collision with first end point, 3: collision with second end point]
    float x - x component of the post collision position
    float y - y component of the post collision position
    float vx - x component of the post collision velocity
    float vy - y component of the post collision velocity
    """
    collision_type = 0
    new_x = 0
    new_y =0
    new_vx = 0
    new_vy=0
    # starting velocity
    v_1 = np.array([vx_1, vy_1])
    
    # vectors from ev to endpoints of wall
    ev_p1x = cell.p1[0] - future_x
    ev_p1y = cell.p1[1] - future_y
    ev_p2x = cell.p2[0] - future_x
    ev_p2y = cell.p2[1] - future_y
    if debug:
        print(' >> cell_collision_recomputation()')
        print(f'    >> Initial velocities: {v_1}, wall/cell direction: {cell.direction[0], cell.direction[1]}')
        print(f"  >>>>  ev_p1x: {ev_p1x:+.6f}, ev_p1y: {ev_p1y:+.6f}")
        print(f"  >>>>  ev_p2x: {ev_p2x:+.6f}, ev_p2y: {ev_p2y:+.6f}")
    
    proj1 = projection(ev_p1x, ev_p1y, cell.direction[0], cell.direction[1])
    proj2 = projection(ev_p2x, ev_p2y, cell.direction[0], cell.direction[1])
        
    cell_dir_unit = cell.direction / (cell.direction**2).sum()**0.5
    
    if debug:
        print(f"  >>>>  projection1: {proj1:+.6f}")
        print(f"  >>>>  projection2: {proj2:+.6f}")
        print(f"  >>>>  cell_dir_unit: {cell_dir_unit}")
    
    #// Perpendicular distance vector from the object to the wall
    perp_dist_x = ev_p1x + cell_dir_unit[0] * -proj1
    perp_dist_y = ev_p1y + cell_dir_unit[1] * -proj1
    perp_dist_length = vlength(perp_dist_x, perp_dist_y)
    perp_dist = add_scaled(ev_p1x, ev_p1y, cell_dir_unit[0], cell_dir_unit[1], -proj1)
    
    test = (abs(proj1) < cell.direction_length) and (abs(proj2) < cell.direction_length)
    if debug:
        print(f'  >>>>  perp_dist_length: {perp_dist_length:+.6f}, perp_dist_x: {perp_dist_x:+.6f}, perp_dist_y: {perp_dist_y:+.6f} | perp_dist: ({perp_dist[0]:+.6f}, {perp_dist[1]:+.6f})')
        print(f'  >>>>  Should we test for a collision? {test}')
        
    if perp_dist_length < agent_radius_um and test:
        collision_type = 1
        # var angle = Vector2D.angleBetween(obj.velo2D, wdir);
        
        dot_product = (dotprod(vx_1, vy_1, cell.direction[0], cell.direction[1]) / (vlength(vx_1, vy_1) * cell.direction_length))
        angle = np.arccos(dot_product)
        if debug:
            print(f'  >>>> Solving collision straight with the cell')
            print(f'    >> dot_product: {dot_product:.6f}, angle: {angle}')
        
        normal_dot_evVelocity = dotprod(cell.unit_normal[0], cell.unit_normal[1], vx_1, vy_1)
        normal_x = cell.unit_normal[0]
        normal_y = cell.unit_normal[1]
        if normal_dot_evVelocity > 0:
            normal_x *= -1.0
            normal_y *= -1.0
        
        deltaS = (agent_radius_um + dotprod(perp_dist_x, perp_dist_y, normal_x, normal_y)) / np.sin(angle)
        if debug:
            print(f'    >> normal_dot_evVelocity: {normal_dot_evVelocity}, new normal (x: {normal_x}, y: {normal_y})')
            print(f'    >> deltaS: {deltaS}')
        
        # 4. estimate what was the displacement, so we compute a vector parallel to 'the velocity vector of this timestep' and of length deltaS
        displacement = parallel(v_1, deltaS, debug)

        # 5. update position by subtracting the displacement
        new_x = future_x - displacement[0]
        new_y = future_y - displacement[1]
        if debug:
            print(f'    >> displacement: (x:{displacement[0]}, y:{displacement[1]}) <---- Error here?')
            print(f'    >> new_x: {new_x}, new_y: {new_y}')
        
        # the correction factor was skipped but in the original implementation it is computes as follows
        # var vcor = 1-acc.dotProduct(displ)/obj.velo2D.lengthSquared();
        #new_velocity_x = vx_1  # *vcor;
        #new_velocity_y = vy_1  # *vcor;
        
        # 6. decompose the velocity
        # velocity vector component perpendicular to wall just before impact
        # var normalVelo = dist.para(Velo.projection(dist));
        velocityProjection = projection(vx_1, vy_1, perp_dist_x, perp_dist_y)
        normalVelocity = parallel(perp_dist, velocityProjection, debug)
        if debug:
            print(f'    >> velocityProjection:{velocityProjection}, normalVelocity:{normalVelocity}')
        
        # velocity vector component parallel to wall; unchanged by impact
        # var tangentVelo = Velo.subtract(normalVelo);
        tangentVelocity_x = vx_1 - normalVelocity[0]
        tangentVelocity_y = vy_1 - normalVelocity[1]
        if debug:
            print(f'    >> tangentVelocity_x:{tangentVelocity_x}, tangentVelocity_y:{tangentVelocity_y}')
        
        # velocity vector component perpendicular to wall just after impact
        # obj.velo2D = tangentVelo.addScaled(normalVelo, -vfac);
        new_vx = tangentVelocity_x + (-normalVelocity[0])
        new_vy = tangentVelocity_y + (-normalVelocity[1])
        if debug:
            print(f'    >> new_vx: {new_vx}, new_vy: {new_vy}')
    else:
        if debug:
            print('  >>>>  No direct collision with the wall')
        pass
    if vlength(ev_p1x, ev_p1y) < agent_radius_um:
        collision_type = 2
        
        if debug:
            print(f'  >>>>  Collision with 1st edge? vlength(ev_p1x, ev_p1y): {vlength(ev_p1x, ev_p1y):+.6f} < radius? {vlength(ev_p1x, ev_p1y) < agent_radius_um}')
    else:
        if debug:
            print('  >>>>  No collision with 1st edge')
    if vlength(ev_p2x, ev_p2y) < agent_radius_um:
        collision_type = 3
        if debug:
            print(f'  >>>>  Collision with 2nd edge? vlength(ev_p2x, ev_p2y): {vlength(ev_p2x, ev_p2y):+.6f} < radius? {vlength(ev_p2x, ev_p2y) < agent_radius_um}')
    else:
        if debug:
            print('  >>>>  No collision with 2nd edge')
            
    return [collision_type, new_x, new_y, new_vx, new_vy]

In [7]:
def ev_collision_validation(ev1, ev2, debug=False):
    """
    Expects two objects
    Checks if a collision would be possible between EV1 and EV2. Both EVs are dictionaries of attributes
    """
    print('----------\n ev_collision_validation')
    # check the distance between their centers
    min_distance = ev1.radius_um + ev2.radius_um
    dist = ev1.location() - ev2.location()
    dist_length = vlength(dist[0], dist[1])
    print(f'min_distance:{min_distance}, dist:{dist}, dist_length:{dist_length}')
    if dist_length < min_distance:
        print('COLLISION')
        print(f'starting pos ev1 [{ev1.x_1}, {ev1.y_1}], ev2 [{ev2.x_1}, {ev2.y_1}]')
        # // normal velocity vectors just before the impact
        # var normalVelo1 = ball1.velo2D.project(dist);
        # var normalVelo2 = ball2.velo2D.project(dist);
        ev1_velo = np.array([ev1.vx, ev1.vy])
        ev2_velo = np.array([ev2.vx, ev2.vy])
        normal_velocity1 = project(ev1_velo, dist)
        normal_velocity2 = project(ev2_velo, dist)
        print(f'ev1_velo:{ev1_velo}, ev2_velo:{ev2_velo}, normal_velocity1:{normal_velocity1}, normal_velocity2:{normal_velocity2}')
        print(f'ev1_unit:{ ev1.vx / vlength(ev1.vx, ev1.vy)},{ ev1.vy / vlength(ev1.vx, ev1.vy)}, ev2_unit:{ ev2.vx / vlength(ev2.vx, ev2.vy)},{ ev2.vy / vlength(ev2.vx, ev2.vy)}')
        dp = normal_velocity1[0] * normal_velocity2[0] + normal_velocity1[1] * normal_velocity2[1]
        if dp < 0:
            print('Oposite directions')
        elif dp > 0:
            print('SAME DIRECTIONS')
        else:
            print('ORTHOGONAL!!!!!')
        
        # // tangential velocity vectors
        # var tangentVelo1 = ball1.velo2D.subtract(normalVelo1);
        # var tangentVelo2 = ball2.velo2D.subtract(normalVelo2);
        tangent_velocity1 = ev1_velo - normal_velocity1
        tangent_velocity2 = ev2_velo - normal_velocity2
        print(f'tangent_velocity1:{tangent_velocity1}, tangent_velocity2:{tangent_velocity2}')
        
        # // move particles so that they just touch	
        # var L = ball1.radius + ball2.radius-dist.length();
        # var vrel = normalVelo1.subtract(normalVelo2).length();
        # ball1.pos2D = ball1.pos2D.addScaled(normalVelo1,-L/vrel);
        # ball2.pos2D = ball2.pos2D.addScaled(normalVelo2,-L/vrel);
        L = min_distance - dist_length
        normal_velo_subtracted = normal_velocity1 - normal_velocity2
        vrel = vlength(normal_velo_subtracted[0], normal_velo_subtracted[1])
        # (src_x, src_y, vec_x, vec_y, scale):
        ev1_loc = ev1.location()
        ev2_loc = ev2.location()
        if dp > 0:
            v1mag = np.sqrt(ev1.vx*ev1.vx+ev1.vy*ev1.vy)
            v2mag = np.sqrt(ev2.vx*ev2.vx+ev2.vy*ev2.vy)
            print(f'ev1 mass:{ev1.mass_ag} vel:{v1mag}, ev2 mass:{ev2.mass_ag} mag:{v2mag}')
            if ev1.mass_ag > ev2.mass_ag:
                if v1mag > v2mag:
                    print("EV1 heavier and faster")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], L/vrel)
                else:
                    print("EV1 heavier but slower")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
            else:
                if v1mag > v2mag:
                    print("EV2 heavier but slower")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
                else:
                    print("EV2 heavier and faster")
                    new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
                    new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], L/vrel)
        else:
            new_pos1 = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
            new_pos2 = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
        new_pos1_old = add_scaled(ev1_loc[0], ev1_loc[1], normal_velocity1[0], normal_velocity1[1], -L/vrel)
        new_pos2_old = add_scaled(ev2_loc[0], ev2_loc[1], normal_velocity2[0], normal_velocity2[1], -L/vrel)
        print(f'normal_velo_subtracted:{normal_velo_subtracted}')
        print(f'L:{L}, vrel:{vrel}, fac:{L/vrel}')
        
        # // normal velocity components after the impact
        # var m1 = ball1.mass;
        # var m2 = ball2.mass;
        # var u1 = normalVelo1.projection(dist);
        # var u2 = normalVelo2.projection(dist);
        # var v1 = ((m1-m2)*u1+2*m2*u2)/(m1+m2);
        # var v2 = ((m2-m1)*u2+2*m1*u1)/(m1+m2);
        u1 = projection(normal_velocity1[0], normal_velocity1[1], dist[0], dist[1])
        u2 = projection(normal_velocity2[0], normal_velocity2[1], dist[0], dist[1])
        #v1 = ((ev1.mass_kg - ev2.mass_kg)*u1+2*ev2.mass_kg*u2) / (ev1.mass_kg + ev2.mass_kg)
        #v2 = ((ev2.mass_kg - ev1.mass_kg)*u2+2*ev1.mass_kg*u1) / (ev1.mass_kg + ev2.mass_kg)
        v1 = ((ev1.mass_ag - ev2.mass_ag)*u1+2*ev2.mass_ag*u2) / (ev1.mass_ag + ev2.mass_ag)
        v2 = ((ev2.mass_ag - ev1.mass_ag)*u2+2*ev1.mass_ag*u1) / (ev1.mass_ag + ev2.mass_ag)
        print(f'u1:{u1}, u2:{u2}, v1:{v1}, v2:{v2}')
        
        # // normal velocity vectors after collision
        # normalVelo1 = dist.para(v1);
        # normalVelo2 = dist.para(v2);
        normal_velocity1 = parallel(dist, v1)
        normal_velocity2 = parallel(dist, v2)
        print(f'normal_velocity1:{normal_velocity1}, normal_velocity2:{normal_velocity2}')
        
        # // final velocity vectors after collision
        # ball1.velo2D = normalVelo1.add(tangentVelo1);
        # ball2.velo2D = normalVelo2.add(tangentVelo2);
        new_vel1 = normal_velocity1 + tangent_velocity1
        new_vel2 = normal_velocity2 + tangent_velocity2
        print(f'new_pos1:{new_pos1}, before_fix:{new_pos1_old}, new_vel1:{new_vel1}, new vlength(new_vel1):{vlength(new_vel1[0], new_vel1[1])}')
        print(f'new_pos2:{new_pos2}, before_fix:{new_pos2_old}, new_vel2:{new_vel2}, new vlength(new_vel2):{vlength(new_vel2[0], new_vel2[1])}')
        print(f'new distance: {np.sqrt((new_pos1[0]-new_pos2[0])**2 + (new_pos1[1]-new_pos2[1])**2)}, old distance: {np.sqrt((new_pos1_old[0]-new_pos2_old[0])**2 + (new_pos1_old[1]-new_pos2_old[1])**2)}')
        print(f'Displacement from starting pos to new pos. With new method: {np.sqrt((new_pos1[0]-ev1.location()[0])**2 + (new_pos1[1] * ev1.location()[1])**2)}, old method:{np.sqrt((new_pos1_old[0]-ev1.location()[0])**2 + (new_pos1_old[1] * ev1.location()[1])**2)}')
        print(f'Displacement from starting pos to new pos. With new method: {np.sqrt((new_pos2[0]-ev2.location()[0])**2 + (new_pos2[1] * ev2.location()[1])**2)}, old method:{np.sqrt((new_pos2_old[0]-ev2.location()[0])**2 + (new_pos2_old[1] * ev2.location()[1])**2)}')
        return [True, new_pos1, new_pos2, new_vel1, new_vel2]
    return [False, 0, 0, 0, 0]

In [7]:
def cf(val):
    return f"{val:>12.7f}" if val else '  --.-------'

def fetch_close_elements_data(ev):
    """
    Reads the info of the agents identified during simulation as closest to each EV agent
    """
    close = {}
    cl_se_id = cl_ci_id = cl_ev_id = -1
    
    cl_se_id = ev['closest_secretory_cell_id']
    cl_ci_id = ev['closest_ciliary_cell_id']
    cl_ev_id = ev['closest_ev_id']
    
    cl_d = 0
    if cl_se_id > -1:
        close = {'element':'secretory', 'id':cl_se_id,
                           'distance':ev['closest_secretory_cell_distance']}
        
    if cl_ci_id > -1:
        close = {'element':'ciliary', 'id': cl_ci_id,
                           'distance':ev['closest_ciliary_cell_distance']}
        
    if cl_ev_id > -1:
        close = {'element':'ev', 'id': cl_ev_id,
                           'distance':ev['closest_ev_distance']}
    
    return close

def print_close_elements_msg(close_data):
    print('Having ', end='')
    if len(close_data) > 0:
        print(close_data['element'], close_data['id'], 'at distance', close_data['distance'])
    else:
        print('no close elements')

def load_error_scenario(scenario_description, previous=1, full_output=False):
    """
    This function loads the whole data for each iteration starting from 'previous' iterations 
    before scenario_description['collision_at'] and up to scenario_description['collision_at'].
    The data is returned as a dictionary containing per iteration:
    {'env':environment, 'secretory':secretory, 'ciliary':ciliary,
    'evs':evs, 'evs_df':evs_df, 'evs_with_errors':evs_with_errors,
                      'close_elements':dict()}
    """
    data = dict()
    
    [itno, environment, secretory, ciliary, evs, evs_df, target_evs_with_errors, *grids] = load_system_state(scenario_description['base_path'], str(scenario_description['collision_at'])+'.xml')
    grids = grids[0]
    data['last'] = itno
    data['first'] = itno - previous
    
    print('Starting iteration:', itno)
    print('Evs with errors:',len(target_evs_with_errors['objects']), 'these are:')
    print('load_error_scenario.1 len(grids):',len(grids))
    
    for err_id in range(len(target_evs_with_errors['objects'])):
        print('  #', target_evs_with_errors['objects'][err_id]._id, ' problematic parameters:', 
              target_evs_with_errors['objects'][err_id].errors_parsing, '@ iteration', itno)
    
    print('Will analysie',previous,'earlier iteration(s), starting from',data['first'])
    
    # at this point we know what elements are close to the offending EVs at the last step of the simulation
    # we need to scan for close elements in previous iterations which could have originated the problem
    print('Loading data for: [',end='')
    for pidx in range(data['last'] - previous, data['last'] + 1, 1):
        [itno, environment, secretory, ciliary, evs, evs_df, evs_with_errors, *grids] = load_system_state(scenario_description['base_path'], str(pidx)+'.xml')
        grids = grids[0]
        print('load_error_scenario.2 len(grids):',len(grids))
        data[pidx] = {'env':environment, 'secretory':secretory, 'ciliary':ciliary,
                      'evs':evs, 'evs_df':evs_df, 'evs_with_errors':evs_with_errors,
                      'close_elements':dict(), 'grid_secretory': grids[0], 'grid_ciliary': grids[1], 'grid_all': grids[2], 'grid_evs':grids[3]}
        
        
        # now we search for close elements to the EVs of our interest in the current iteration
        # our target EVs could be found in the evs or evs_with_errors
        for ev_err in target_evs_with_errors['objects']:
            for ev in evs['dictionaries']:
                if ev_err._id == ev['id']:
                    close_e_data = fetch_close_elements_data(ev)
                    if len(close_e_data) > 1:
                        data[pidx]['close_elements'][ev_err._id] = copy.deepcopy(close_e_data)
            for evrr in evs_with_errors['dictionaries']:
                if ev_err._id == evrr['id']:
                    close_err_data = fetch_close_elements_data(evrr)
                    if len(close_err_data) > 1:
                        data[pidx]['close_elements'][ev_err._id] = copy.deepcopy(close_err_data)
                    
        print(pidx,end='')
        print(' close elems:', data[pidx]['close_elements'], end=', ')
    else:
        print('] \nDone loading')
    
    return data

def traverse_and_test(data, f=None):
    """
    Shows data for each offending ev and their closest elements per iteration.
    If a testing function is provided, it will be executed for each element in the list of evs_with_errors
    However, any tests here depend on having elements in evs_with_errors. 
    Elements will only exist in evs_with_errors if the iteration data was saved/exported correctly (including NaNs or Infs).
    Otherwise, testing must be executed using manually configured test functions
    """
    for i in range(len(data[data['last']]['evs_with_errors']['objects'])):
        o_w_err = data[data['last']]['evs_with_errors']['objects'][i]
        d_w_err = data[data['last']]['evs_with_errors']['dictionaries'][i]
        
        # pidx - number of iteration
        for pidx in range(data['first'], data['last'], 1):
            print('@',pidx)
            for idx in range(len(data[pidx]['evs']['objects'])):
                o_ev = data[pidx]['evs']['objects'][idx]
                d_ev = data[pidx]['evs']['dictionaries'][idx]
                
                if o_ev._id == o_w_err._id:
                    o_ev.print_resume(cf)
                    
                    # check if this EV has close elements in the iteration?
                    if o_ev._id in data[pidx]['close_elements']:
                        close_element_data = data[pidx]['close_elements'][o_ev._id]
                        print_close_elements_msg(close_element_data)
                        
                        if f: # we need to fetch the elememt and test
                            if close_element_data['element'] == 'secretory':
                                for idx in range(len(data[pidx]['secretory']['objects'])):
                                    o_sec = data[pidx]['secretory']['objects'][idx]
                                    d_sec = data[pidx]['secretory']['dictionaries'][idx]
                                    if o_sec._id == close_element_data['id']:
                                        f(o_ev, o_sec)
        else:
            print('@',data['last'])
            for idx in range(len(data[data['last']]['evs_with_errors']['objects'])):
                o_ev = data[data['last']]['evs_with_errors']['objects'][idx]
                d_ev = data[data['last']]['evs_with_errors']['dictionaries'][idx]
                if o_ev._id == o_w_err._id:
                    o_ev.print_resume(cf)
                    
                    if o_ev._id in data[data['last']]['close_elements']:
                        close_element_data = data[data['last']]['close_elements'][o_ev._id]
                        print_close_elements_msg(close_element_data)
                        if f:
                            if close_element_data['element'] == 'secretory':
                                for idx in range(len(data[pidx]['secretory']['objects'])):
                                    o_sec = data[data['last']]['secretory']['objects'][idx]
                                    d_sec = data[data['last']]['secretory']['dictionaries'][idx]
                                    if o_sec._id == close_element_data['id']:
                                        f(o_ev, o_sec)

In [1]:
def point_to_line_segment_distance(x, y, x1, y1, x2, y2):
    """
    Replicates the functions used during simulation.
    """
    A = x - x1
    B = y - y1
    C = x2 - x1
    D = y2 - y1

    dot = A * C + B * D
    len_sq = C * C + D * D
    param = -1
    xx = 0.
    yy = 0.

    if len_sq != 0: # in case of 0 length line
        param = dot / len_sq

    if param < 0: # the closest point is p1, point not projecting on segment
        xx = x1
        yy = y1
    elif param > 1: # the closest point is p2, point not projecting on segment
        xx = x2
        yy = y2
    else:  # interpolate to find the closes point on the line segment
        xx = x1 + param * C
        yy = y1 + param * D

    # dx / dy is the vector from p0 to the closest point on the segment
    dx = x - xx
    dy = y - yy
    d = np.sqrt(dx * dx + dy * dy)
    
    msg = f'point_to_line_segment_distance(x:{x}, y:{y}, x1:{x1}, y1:{y1}, x2:{x2}, y2:{y2})\n' \
        f'  >>>  A:{A} B:{B} C:{C} D:{D} \n' \
        f'  >>>  dot(A*C+B*D):{dot}, len_sq: {len_sq} \n' \
        f'  >>>  param: {param}, d:{d:.6f}, xx:{xx}, yy:{yy}, dx:{dx:.6f}, dy:{dy:.6f}'
    
    return [param, d, xx, yy, msg]

# object based comparisons
def test_ev_object_and_cell_object_collision(ev, cell, prev=False, hypothetical=False):
    def print_result(res):
        sep = res[1] - ev.radius_um
        if res[0] < 0:
            print('Projecting outside the segment. The closest point is P1 at', res[1], 'Considering the EV radius, the actual separation is',sep)
        elif res[0] > 1:
            print('Projecting outside the segment. The closest point is P2 at', res[1], 'Considering the EV radius, the actual separation is',sep)
        else:
            print('Projecting on the segment, at distance',res[1],'Considering the EV radius, the separation is',sep)
            print('risk of inestability?', sep < -1e-6)
            print('The point\'s projection intersects the segment at',res[2:4])
            
    loc = None
    if ev.vx and ev.vy:
        dp = dotprod(ev.vx, ev.vy, cell.unit_normal[0], cell.unit_normal[1])
        print('Dot product', dp, 'will check distance' if dp <= 0 else 'will NOT check distance')
        loc = np.array([ev.x_1, ev.y_1])
        if ev.vx and ev.vy:
            nloc = loc + np.array([ev.vx, ev.vy]) * 0.005
            print('nloc',nloc)
        else:
            print('skipping location check')
    else:
        print("Skipping DP test")

    if prev:
        print('Checking with previous position')
        res = point_to_line_segment_distance(ev.x_1, ev.y_1, cell.p1[0], cell.p1[1], cell.p2[0], cell.p2[1])
        print_result(res)
        
    print('Checking with current position')
    res = point_to_line_segment_distance(ev.x, ev.y, cell.p1[0], cell.p1[1], cell.p2[0], cell.p2[1])
    print_result(res)
    
    if res[1] < ev.radius_um + 1e-6:
        if res[1] < ev.radius_um:
            print('EV colliding')
        else:
            print('Potential EV collision due to FP errors')
    else:
        print('EV not yet colliding')
    
    # check collision

    if loc is not None and hypothetical:
        print('Checking hypothetical position')
        res = point_to_line_segment_distance(loc[0], loc[1], cell.p1[0], cell.p1[1], cell.p2[0], cell.p2[1])
        print_result(res)

def test_point_to_line_segment_distance_w_dictionaries(ev, cell):
    def print_result(res):
        sep = res[1] - ev['radius_um']
        if res[0] < 0:
            print('    Projecting outside the segment. The closest point is P1 at', res[1], 'Considering the EV radius ({ev[\'radius_um\']}), the actual separation is',sep)
        elif res[0] > 1:
            print('    Projecting outside the segment. The closest point is P2 at', res[1], 'Considering the EV radius ({ev[\'radius_um\']}), the actual separation is',sep)
        else:
            print('    Projecting on the segment, at distance',res[1],'Considering the EV radius, the separation is',sep)
            print('    risk of inestability?', sep < 1e-5, 'separation is', sep)
            print('    The point\'s projection intersects the segment at', res[2:4])

    # checking distance to segment from the initial position
    print('  Checking distance to target segment from initial position (For debugging purposes only)')
    res = point_to_line_segment_distance(ev['x_1'], ev['y_1'], cell['p1_x'], cell['p1_y'], cell['p2_x'], cell['p2_y'])
    print_result(res)
    
    # check collision
    if 'debugOriginalX' in ev and 'debugOriginalY' in ev:
        print('  Checking the distance to target segment from the originally computed position (To verify any missed collision)')
        res = point_to_line_segment_distance(ev['debugOriginalX'], ev['debugOriginalY'], cell['p1_x'], cell['p1_y'], cell['p2_x'], cell['p2_y'])
        print_result(res)
        if res[1] < ev['radius_um'] + 1e-6:
            if res[1] < ev['radius_um']:
                print('  The originally computed position should produce a collision')
            else:
                print('  The originally computed position could produce a collision due to rounding errors')
        else:
            print('  The final position is NOT within collision distance')
        
    else:
        print('  CANNOT check distance to target segment from originally computed position. Really? Ev data:')
        print(ev)

In [13]:
data = load_error_scenario(e1, previous=1, full_output=True)
traverse_and_test(data)

Starting iteration: 256111
Evs with errors: 1 these are:
  # 7462  problematic parameters: ['x', 'y', 'vx', 'vy', 'bm_vx', 'bm_vy', 'debugOriginalX', 'debugOriginalY', 'debugNormalVelX', 'debugNormalVelY', 'debugTangVelX', 'debugTangVelY'] @ iteration 256111
Will analysie 1 earlier iteration(s), starting from 256110
Loading data for: [256110 close elems: {}, 256111 close elems: {}, ] 
Done loading
@ 256110
       x: 411.5399170 y: 432.0410160
     v x: -16.7643300 y:  31.7879770
    bm x:   0.1424660 y:  -0.1108500
   t-1 x: 411.6237490 y: 431.8820800
radius_um 0.132
@ 256111
       x:  --.------- y:  --.-------
     v x:  --.------- y:  --.-------
    bm x:  --.------- y:  --.-------
   t-1 x: 411.5255130 y: 432.0680240
radius_um 0.132


In [8]:
traverse_and_test(data, test_ciliary_cell_collision)

@ 210522
       x: 632.1369630 y: 124.1384350
     v x:  12.6187300 y:  -2.2578620
    bm x:   0.0392700 y:  -0.0503560
   t-1 x: 632.0738530 y: 124.1497270
radius_um 0.054
@ 210523
       x: 632.1459960 y: 124.1367720
     v x:  --.------- y:  --.-------
    bm x:  -0.0067360 y:  -0.0637410
   t-1 x: 632.1369630 y: 124.1384350
radius_um 0.054
Having secretory 931 at distance 0.0
Skipping DP test
Checking with current position
Projecting on the segment, at distance 0.054016000000046915 Considering the EV radius, the separation is 1.6000000046915985e-05
risk of inestability? False
The point's projection intersects the segment at [632.200012, 124.136772]
EV not yet colliding


In [78]:
data = load_error_scenario(e2, previous=4, full_output=True)

Starting iteration: 243646
Evs with errors: 1 these are:
  # 7270  problematic parameters: ['x', 'y', 'x_1', 'y_1'] @ iteration 243646
Will analysie 4 earlier iteration(s), starting from 243642
Loading data for: [243642 close elems: {}, 243643 close elems: {}, 243644 close elems: {}, 243645 close elems: {7270: {'element': 'secretory', 'id': 3048, 'distance': 0.141149}}, 243646 close elems: {}, ] 
Done loading


In [79]:
traverse_and_test(data, test_ciliary_cell_collision)

@ 243642
       x: 301.0156860 y: 545.9846190
     v x:   0.0031880 y:  -0.0032270
    bm x:  -0.0000160 y:   0.0000160
   t-1 x: 301.0156560 y: 545.9846190
radius_um 0.154
@ 243643
       x: 301.0157170 y: 545.9846190
     v x:   0.0031810 y:  -0.0032050
    bm x:  -0.0000070 y:   0.0000220
   t-1 x: 301.0156860 y: 545.9846190
radius_um 0.154
@ 243644
       x: 301.0157470 y: 545.9846190
     v x:   0.0031980 y:  -0.0032200
    bm x:   0.0000170 y:  -0.0000150
   t-1 x: 301.0157170 y: 545.9846190
radius_um 0.154
@ 243645
       x: 294.6236570 y: 552.3585820
     v x:   0.0032080 y:  -0.0032170
    bm x:   0.0000190 y:   0.0000120
   t-1 x: 301.0157470 y: 545.9846190
radius_um 0.154
Having secretory 3048 at distance 0.141149
Dot product 6.363962999999737e-06 will NOT check distance
nloc [301.01576304 545.98460291]
Checking with current position
Projecting outside the segment. The closest point is P1 at 7.45089984488777 Considering the EV radius, the actual separation is 7.2968998448877

In [44]:
"""
DATA dictionaries contain: [
        'env': simulation parameters at this time during simulation, 
        'secretory':['objects','dictionaries'], 
        'ciliary', 
        'evs',
        'evs_df' - The EV collection in Pandas data frame format
        'evs_with_errors' - a list of dictionaries parsed from XML,
        'close_elements' - a list of nearby elements (as read from the XML?)
        'grid_secretory', 'grid_ciliary', 'grid_all', 'grid_evs' - GridStorages for these objects
"""

def fetch_element_by_id_from_dictionaries_list(dict_list, element_id):
    for element in dict_list:
        if element['id'] == element_id:
            print('Element found in dictionary list')
            return element

def fetch_element_by_id_from_objects_list(objects_list, element_id):
    for element in objects_list:
        if element._id == element_id:
            print('Element found in objects list')
            return element

def fetch_ev_from_data(data, idx, ev_id):
    # fetch ev from dictionary list
    if len(data[idx]['evs_with_errors']) > 0:
        ev_d = fetch_element_by_id_from_dictionaries_list(data[idx]['evs_with_errors']['dictionaries'], ev_id)
        if not ev_d:
            ev_d = fetch_element_by_id_from_dictionaries_list(data[idx]['evs']['dictionaries'], ev_id)
    else:
        ev_d = fetch_element_by_id_from_dictionaries_list(data[idx]['evs']['dictionaries'], ev_id)
    
    if not ev_d:
        raise ValueError('EV', ev_id, 'not found in iteration', idx)
    return ev_d
    
def fetch_cell_from_data(data, idx, cell_id, cell_type):
    print('Fetching',cell_type,'cell')
    cell_d = fetch_element_by_id_from_dictionaries_list(data[idx][cell_type]['dictionaries'], cell_id)
    cell_o = fetch_element_by_id_from_objects_list(data[idx][cell_type]['objects'], cell_id)
    if cell_d:
        print(cell_type,'cell found')
    else:
        raise ValueError(cell_type, 'cell', cell_id, 'not found in iteration', idx)
    return cell_d, cell_o    

def wall_segment_test(data, experiment_params):
    """
    Performs a forced check using the debugging values in the EV having the
    id==ev_id (These are non-standard ev attributes)
    
    For this comparison we must use a GridStorage object
    We must compare the given EV directly against:
        - The offending wall in the current iteration
        - The nearest neighbouring walls (within 2x5um radius?) in current iteration
        - The offending wall in the previous iteration
        - The nearest neighbouring walls (within 2x5um radius?) in the previous iteration
    Input:
    data - Dictionaries of data per iteration
    
    For fetching the offending/target EV, we need to check its presence in the evs_with_errors
    list before attempting to do so from any of the more 'standard' collections.
    
    For simplicity, this comparison will be carried using the dictionary based
    representation of the agents
    """
    #idx = data['last']
    iters = [i for i in range(data['last'], data['first']-1, -1)]
    print('iterations to check',iters)
    for idx in iters:
        print('Processing iter', idx)
        print('Fetching EV')
        ev_d = fetch_ev_from_data(data, idx, experiment_params['ev_id'])

        if experiment_params['secretory_id']:
            cell_d, cell_o = fetch_cell_from_data(data, idx, experiment_params['secretory_id'], 'secretory')
        elif experiment_params['ciliary_id']:
            cell_d, cell_o = fetch_cell_from_data(data, idx, experiment_params['ciliary_id'], 'ciliary')
        else:
            raise ValueError('No Secretory or Ciliary cell specified')

        print('Comparing directly against cell segment...')
        # checking distance to segment from intended next position
        #test_ev_dictionary_and_cell_dictionary_collision(ev_d, cell_d)
        test_point_to_line_segment_distance_w_dictionaries(ev_d, cell_d)
        #checking distance to segment from starting position
        
        print('Comparing against other cell segments in the vicinity')
        # fetch the segments in the vicinity
        
            # checking distance to segment from intended next position
        
            #checking distance to segment from starting position

In [15]:
data[256111]['evs_with_errors']

{'objects': [<ev_model.elements.EV at 0x7fcdec536160>],
 'dictionaries': [{'id': 7462,
   'errors_parsing': ['x',
    'y',
    'vx',
    'vy',
    'bm_vx',
    'bm_vy',
    'debugOriginalX',
    'debugOriginalY',
    'debugNormalVelX',
    'debugNormalVelY',
    'debugTangVelX',
    'debugTangVelY'],
   'x': None,
   'y': None,
   'z': 0.0,
   'x_1': 411.525513,
   'y_1': 432.068024,
   'vx': None,
   'vy': None,
   'bm_vx': None,
   'bm_vy': None,
   'mass_ag': 1008.288879,
   'radius_um': 0.132,
   'diffusion_rate_um': 1.624068,
   'diff_rate_um_x_twice_dof': 6.496274,
   'closest_ev_id': -1,
   'closest_ev_distance': 100.0,
   'closest_secretory_cell_id': -1,
   'closest_secretory_cell_distance': 100.0,
   'closest_ciliary_cell_id': -1,
   'closest_ciliary_cell_distance': 100.0,
   'age': 990.745972,
   'time_in_initial_state': 1.464828,
   'velocity_ums': 0.180226,
   'colInfVnorm': 0.0,
   'colInfVrel': 0.0,
   'colInfOverlap': 0.0,
   'colInfCorrectionFactor': 0.0,
   'colInfAuxF

In [55]:
data = load_error_scenario(e3, previous=1, full_output=True)
traverse_and_test(data)

read_xml len(grids): 4
load_system_state len(grids): 4
Starting iteration: 256111
Evs with errors: 0 these are:
load_error_scenario.1 len(grids): 4
Will analysie 1 earlier iteration(s), starting from 256110
Loading data for: [read_xml len(grids): 4
load_system_state len(grids): 4
load_error_scenario.2 len(grids): 4
256110 close elems: {}, read_xml len(grids): 4
load_system_state len(grids): 4
load_error_scenario.2 len(grids): 4
256111 close elems: {}, ] 
Done loading


In [61]:
#if 'wall' in e3:
    # fetch wall 3 from curr and prev iterations, compare distance
print('checking wall', e3['secretory_id'])
wall_segment_test(data, e3)

checking wall 2329
iterations to check [256111, 256110]
Processing iter 256111
Fetching EV
Element found in dictionary list
Fetching secretory cell
Element found in dictionary list
Element found in objects list
secretory cell found
Comparing directly against cell segment...
  Checking distance to target segment from initial position
    Projecting on the segment, at distance 0.1319880000000353 Considering the EV radius, the separation is -1.1999999964706909e-05
    risk of inestability? True
    The point's projection intersects the segment at [411.525513, 432.200012]
  Will be checking the distance to target segment from the originally computed position
    Projecting on the segment, at distance 0.027039000000002034 Considering the EV radius, the separation is -0.10496099999999797
    risk of inestability? True
    The point's projection intersects the segment at [411.43988, 432.200012]
  Checking the distance to target segment from the final position in the current iteration. After a

In [23]:
data[256111]['evs']['dictionaries']

[{'id': 0,
  'p1_x': 774.200012,
  'p1_y': 432.0,
  'p2_x': 774.200012,
  'p2_y': 430.0,
  'x': 774.200012,
  'y': 431.0,
  'z': 0.0,
  'direction_x': 0.0,
  'direction_y': -2.0,
  'direction_x_unit': 0.0,
  'direction_y_unit': -1.0,
  'direction_length': 2.0,
  'normal_x': -2.0,
  'normal_y': -0.0,
  'unit_normal_x': -1.0,
  'unit_normal_y': -0.0,
  'normal_length': 2.0,
  'time_to_next_secretion_attempt': 9.094934,
  'probability_of_secretion': 0.0,
  'source_points': 3,
  'source_points_xs': [774.200012,
   774.200012,
   774.200012,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  'source_points_ys': [431.5,
   431.0,
   430.5,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  'last_source_point_secreting': 0},
 {'id': 4845,
  'p1_x': 772.200012,
  'p1_y': 434.0,
  'p2_x': 772.200012,
  'p2_y': 433.0,
  'x': 772.200012,
  'y': 433.5,
  'z': 0.0,
  'direction_x': 0.0,
  'direction_y': -1.0,
  'direction_x_unit': 0.0,
  'direction_y_unit': -1.0,
  '

Element found in evs_with_errors list


# Standard workflow

1. kill_ev
1. *state resetting*
  1. reset_state
  1. initial_to_default
1. brownian_movement_2d
1. *movement*
  1. move
  1. moveInitial
1. *output cell locations*
  1. output_ciliary_cell_location
  1. output_secretory_cell_location
1. test_secretory_cell_collision
1. test_ciliary_cell_collision
1. secretory_cell_collision_resolution
1. ciliary_cell_collision_resolution
1. secrete_ev
1. output_location_ev_initial
1. test_collision_ev_default_ev_initial
1. output_location_ev_default
1. test_collision_ev_default_ev_default